#**This notebook explores the data to identify which attributes to use for the weighting algorithm**





#Installing the necessary packages

In [1]:
!pip install osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 95 kB 3.0 MB/s 
     |████████████████████████████████| 1.0 MB 60.9 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 11.2 MB 29.3 MB/s 
     |████████████████████████████████| 6.3 MB 38.8 MB/s 
     |████████████████████████████████| 1.0 MB 39.4 MB/s 
     |████████████████████████████████| 16.7 MB 271 kB/s 
     |████████████████████████████████| 957 kB 46.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


# Importing the libraries


In [1]:
import osmnx as ox
import networkx as nx
import numpy as np

# Fetching Data from Open Street Maps and making a graph

In [2]:
# fetch osm data for Zurich
# Note: The search term was found using: https://nominatim.openstreetmap.org/ui/search.html
print('[Velosafe] Fetching OSM data')
#graph = ox.graph_from_place('Altstadt, Zurich, District Zurich, Zurich, 8001, Switzerland', network_type="bike") # Fetch only Kreis 1
#graph = ox.graph_from_place('Zurich, District Zurich, Zurich, Switzerland', network_type="bike") # Fetch Zurich
graph = ox.graph_from_address('Bahnhofbrücke, Bahnhofquai, Hochschulen, Altstadt, Zurich, District Zurich, Zurich, 8001, Switzerland', network_type="bike") # Fetch Around Bahnhofbrücke


[Velosafe] Fetching OSM data


#Adding the edge speeds and edge travel times to the graph

In [3]:
# Infer edge speeds and travel times
print('[Velosafe] Inferring travel times')
graph = ox.speed.add_edge_speeds(graph)
graph = ox.speed.add_edge_travel_times(graph)

[Velosafe] Inferring travel times


# Getting the nodes and edges of the graph that is constructed

In [4]:
## Modifying the graph
# Extracting GeoDataFrame from graph
print('[Velosafe] Applying custom weighting')
gdf_nodes, gdf_edges = ox.graph_to_gdfs(graph)

[Velosafe] Applying custom weighting


# Cleaning up the data and inspecting the information included in each edge of the graph

In [5]:
# Cleaning up Dataframe, making sure there are no lists
gdf_edges['osmid'] = gdf_edges['osmid'].apply(lambda x: x[0] if isinstance(x, list) else x)
gdf_edges['maxspeed'] = gdf_edges['maxspeed'].apply(lambda x: x[0] if isinstance(x, list) else x)
gdf_edges['highway'] = gdf_edges['highway'].apply(lambda x: x[0] if isinstance(x, list) else x)
gdf_edges['lanes'] = gdf_edges['lanes'].apply(lambda x: x[0] if isinstance(x, list) else x)
gdf_edges['access'] = gdf_edges['access'].apply(lambda x: x[0] if isinstance(x, list) else x)
gdf_edges['width'] = gdf_edges['width'].apply(lambda x: x[0] if isinstance(x, list) else x)
gdf_edges['tunnel'] = gdf_edges['tunnel'].apply(lambda x: x[0] if isinstance(x, list) else x)
gdf_edges['service'] = gdf_edges['service'].apply(lambda x: x[0] if isinstance(x, list) else x)



# Inspecting information in each edge
print(gdf_edges.columns)

Index(['osmid', 'lanes', 'name', 'highway', 'maxspeed', 'oneway', 'length',
       'speed_kph', 'travel_time', 'geometry', 'access', 'bridge', 'ref',
       'width', 'tunnel', 'service'],
      dtype='object')


# All the code lines ahead were implemented to identify which attribute of the edge has what corresponding value so that an algorithm could be constructed using this

In [6]:
gdf_edges.head(2)

osmid lanes               name   highway maxspeed  \
u       v          key                                                        
4366192 5257663325 0    4572089     4  Stauffacherbrücke  tertiary       50   
        88993792   0    4687935     1      Selnaustrasse  tertiary       50   

                        oneway  length  speed_kph  travel_time  \
u       v          key                                           
4366192 5257663325 0     False   8.273       50.0          0.6   
        88993792   0      True  74.344       50.0          5.4   

                                                                 geometry  \
u       v          key                                                      
4366192 5257663325 0      LINESTRING (8.53133 47.37099, 8.53123 47.37102)   
        88993792   0    LINESTRING (8.53133 47.37099, 8.53135 47.37094...   

                       access bridge  ref width tunnel service  
u       v          key                                          
4366192 5257663325 0      NaN    NaN  NaN   NaN    NaN     NaN  
        88993792   0      NaN    NaN  NaN   NaN    NaN     NaN

In [7]:
gdf_edges.lanes.unique()

array(['4', '1', nan, '2', '3', '5', '6'], dtype=object)

In [ ]:
gdf_edges.highway.unique()

array(['tertiary', 'cycleway', 'residential', 'service', 'living_street',
       'primary', 'pedestrian', 'secondary', 'unclassified', 'path',
       'trunk_link', 'primary_link'], dtype=object)

In [ ]:
gdf_edges.maxspeed.unique()

array(['50', nan, '20', '30', 'walk', '80', '60'], dtype=object)

In [ ]:
gdf_edges.oneway.unique()

array([False,  True])

In [ ]:
gdf_edges.length.unique()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


array([1.04843693e-04, 8.62061313e-04, 6.72846193e-05, ...,
       2.44052556e-04, 4.23287430e-04, 1.24322458e-03])

In [ ]:
gdf_edges.speed_kph.unique()

array([50. , 43.3, 33.3, 43.9, 20. , 30. , 40. , 21.6, 48.1, 25. , 80. ,
       60. , 35. ])

In [ ]:
gdf_edges.travel_time.unique()

array([6.000e-01, 5.400e+00, 5.000e-01, 7.800e+00, 6.400e+00, 8.600e+00,
       5.300e+00, 4.000e-01, 6.500e+00, 8.000e-01, 2.500e+00, 2.200e+00,
       9.000e-01, 5.900e+00, 2.210e+01, 4.200e+00, 1.100e+00, 1.000e+00,
       1.500e+00, 7.300e+00, 4.500e+00, 1.040e+01, 1.320e+01, 7.000e+00,
       5.800e+00, 9.100e+00, 3.200e+00, 2.700e+00, 2.600e+00, 7.000e-01,
       7.200e+00, 8.300e+00, 1.350e+01, 5.600e+00, 5.200e+00, 3.600e+00,
       1.500e+01, 1.150e+01, 6.800e+00, 1.230e+01, 8.400e+00, 1.700e+00,
       6.700e+00, 7.700e+00, 2.400e+00, 3.000e+00, 6.300e+00, 4.600e+00,
       3.900e+00, 1.100e+01, 5.100e+00, 4.000e+00, 4.100e+00, 1.200e+00,
       1.380e+01, 1.270e+01, 1.130e+01, 4.400e+00, 4.800e+00, 3.000e-01,
       6.200e+00, 2.220e+01, 7.100e+00, 5.000e+00, 2.300e+00, 3.700e+00,
       2.900e+00, 1.180e+01, 2.100e+00, 1.400e+00, 9.400e+00, 9.300e+00,
       6.000e+00, 9.000e+00, 1.880e+01, 1.300e+00, 4.900e+00, 8.200e+00,
       2.000e+00, 1.700e+01, 1.010e+01, 1.410e+01, 

In [ ]:
gdf_edges.geometry.unique()

<GeometryArray>
 ...
Length: 2945, dtype: geometry

In [ ]:
gdf_edges.access.unique()

array([nan, 'no', 'destination', 'permissive', 'delivery', 'customers'],
      dtype=object)

In [ ]:
gdf_edges.bridge.unique()

array([nan, 'yes'], dtype=object)

In [ ]:
gdf_edges.ref.unique()

array([nan, '17', 'A1L'], dtype=object)

In [ ]:
gdf_edges.width.unique()

array([nan, '5', '4', '2', '7', '4.3', '3', '4.5', '10', '5.2', '15',
       '11', '1', '12', '6.5', '20', '15.5', '3.5', '1.5', '2.5', '6.6',
       '1.2', '8', '6', '4.9', '3.1', '0.5', '6.1'], dtype=object)

In [ ]:
gdf_edges.tunnel.unique()

array([nan, 'yes', 'building_passage'], dtype=object)

In [ ]:
gdf_edges.service.unique()

array([nan, 'parking_aisle', 'driveway', 'alley', 'emergency_access',
       'drive-through'], dtype=object)